# Exercise 1: House price levels and dispersion

For this exercise, we're using data on around 3,000 observations of house prices and house characteristics from Ames, a small city in Iowa.

1.  Load the Ames housing data set from `ames_houses.csv` located in the `data/` folder.

    To familiarize yourself with the data, report the columns present in the DataFrame and 
    then restrict the data to the columns `SalePrice` and `Neighborhood`.
    
    Check that there are no observations with missing values in the final data set.

2.  Compute the average house price (column `SalePrice`) by neighborhood (column `Neighborhood`). List the three most expensive neighborhoods, for example by using [`sort_values()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html).

    Create a bar chart showing the average sale price for all neighborhoods in descending order.

    *Hint:* You can create the bar chart by either using pandas's 
    [`DataFrame.plot.bar()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html)
    or Matplotlib's
    [`bar()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html).

3.  You want to quantify the price dispersion in each neighborhood. To this end, compute the standard deviation by neighborhood using [`std()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.std.html). Which are the three neighborhoods with the most dispersed prices?

    Create a scatter plot with the average house price on the $x$-axes and the standard deviation on the $y$-axis.
    Is there a relationship between the average house prices and their dispersion within neighborhood?

    *Hint:* You can create the scatter plot by either using pandas's 
    [`DataFrame.plot.scatter()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.scatter.html)
    or Matplotlib's 
    [`scatter()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html).

In [ ]:
import pandas as pd
DATA_PATH = '../../data'
fn = f'{DATA_PATH}/ames_houses.csv'
df = pd.read_csv(fn)
df

In [ ]:
df_subset = df[['SalePrice','Neighborhood']]
missing_counts = df_subset.isnull().sum()
print(missing_counts)
#df_subset = df_subset.dropna()
print(df_subset)

In [ ]:
data = df_subset.groupby('Neighborhood')['SalePrice'].mean()


In [ ]:
top3 = data.sort_values(ascending = False).head()
plt.bar(top3.index,top3.values)
plt.xlim(1,3)

In [ ]:
import matplotlib.pyplot as plt
plt.bar(data.index,data.values)
_ = plt.xticks(rotation=90)
plt.xlabel('Name of Neighborhoods')
plt.ylabel('Average Price of Houses')
plt.title('Ames Housing')

In [ ]:
data1 = df_subset.groupby('Neighborhood')['SalePrice'].std()

In [ ]:
plt.scatter(data1.index,data1.values)
__ = plt.xticks(rotation=90)


***
# Exercise 2: Determinants of house prices

For this exercise, we're using data on around 3,000 observations of house prices and house characteristics from Ames, a small city in Iowa,
to understand how house prices vary with selected house characteristics.

1.  Load the Ames housing data set from `ames_houses.csv` located in the `data/` folder, and keep only the
    columns `SalePrice`, `LotArea`, `YearBuilt`, and `Bedrooms`.

    Restrict your data set to houses with one or more bedrooms and a lot area of at least 100m².

2.  Compute the average year in which a house was built (using the column `YearBuilt`). Create a new column `New` which takes on the value of 1 if the house was built after the average year of construction (_"new"_), and 0 otherwise (_"old"_). 

    What is the average year of construction within these two categories?

3.  Create a new column `Rooms` which categorizes the number of `Bedrooms` into three groups: 1, 2, and 3 or more. You can create these categories using boolean indexing, [`np.where()`](https://numpy.org/doc/2.0/reference/generated/numpy.where.html), pandas's [`where()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.where.html), or some other way.

4.  Compute the mean `SalePrice` within each group formed by `New` and `Rooms` (for a total of 6 different categories) using [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html).

    What is the average price difference between an old house with 1 bedroom and a new house with 3+ bedrooms?

5.  Create a figure with two subplots arranged in two columns. The left column should contain a bar chart showing
    the average house price by number of bed rooms (1, 2, 3+) for _old_ houses, whereas the 
    right column should show the corresponding bars for _new_ houses.

In [ ]:
import pandas as pd
DATA_PATH = '../../data'
fn = f'{DATA_PATH}/ames_houses.csv'
df = pd.read_csv(fn)

In [ ]:
df_subset2 = df[['SalePrice', 'LotArea', 'YearBuilt', 'Bedrooms']]
df_filtered = df_subset2[(df_subset2['Bedrooms'] >= 1) & (df_subset2['LotArea'] >= 100000)]

In [ ]:
avg_yr = int(df['YearBuilt'].mean())
df['New'] = (df['YearBuilt'] == avg_yr).astype(int)

avg_yr_construction = df.groupby('New')['YearBuilt'].mean()
print(avg_yr_construction)


In [ ]:
import numpy as np
df['Rooms'] = df['Bedrooms'].where(df['Bedrooms'] > 3)
df.groupby(['New','Rooms'])['SalePrice'].mean()

In [101]:
df['Rooms'] = np.where(df['Bedrooms'] == 1, 1,
             np.where(df['Bedrooms'] == 2, 2,
             np.where(df['Bedrooms'] >= 3, 3, np.nan)))
df.groupby(['New','Rooms'])['SalePrice'].mean()

New  Rooms
0    1.0      183.016964
     2.0      162.610109
     3.0      187.827464
1    2.0      132.754545
     3.0      143.442857
Name: SalePrice, dtype: float64

***
# Exercise 3: Inflation and unemployment in the US

In this exercise, you'll be working with selected macroeconomic variables for the United States reported at monthly frequency obtained from [FRED](https://fred.stlouisfed.org/).
The data set starts in 1948 and contains observations for a total of 864 months.

1.  Load the data from the file `FRED_monthly.csv` located in the `data/FRED` folder. Print the first 10 observations to get an idea how the data looks like.
    
    Keep only the columns `Year`, `Month`, `CPI`, and `UNRATE`. Moreover, perform this analysis only on observations prior to 1970 and drop the rest.

2.  The column `CPI` stores the consumer price index for the US. You may be more familiar with the concept of inflation, which is the percent change of the CPI relative to the previous period. 
    Create a new column `Inflation` which contains the _annual_ inflation _in percent_ relative to the same month in the previous year by applying 
    [`pct_change()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) to the column `CPI`.

    *Hints:* 
    
    -   Since this is monthly data, you need to pass the arguments `periods=12` to `pct_change()` to get annual percent changes.
    -   You need to multiply the values returned by `pct_change()` by 100 to get percent values.

3.  Compute the average unemployment rate (column `UNRATE`) over the whole sample period. Create a new column `UNRATE_HIGH` that contains an indicator whenever the unemployment rate is above its average value (_"high unemployment period"_). 
    -   How many observations fall into the high- and the low-unemployment periods?
    -   What is the average unemployment rate in the high- and low-unemployment periods?

4.  Compute the average inflation rate for high- and low-unemployment periods. Is there any difference?

5.  Use [`resample()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html) to aggregate
    the inflation data to annual frequency and compute the average inflation within each calendar year.

    -   Report the three years with the highest inflation rates.
    -   Create a plot that shows the average annual inflation over the sample period.

    *Hint:* Use the resampling rule `'YE'` when calling `resample()`.
